# Валидация данных

Пример на основе набора данных об пассажирах Титаника - https://www.kaggle.com/c/titanic

In [2]:
with open("BankChurners.csv") as f:
    for i in range(10):
        print(f.readline())

"CLIENTNUM","Attrition_Flag","Customer_Age","Gender","Dependent_count","Education_Level","Marital_Status","Income_Category","Card_Category","Months_on_book","Total_Relationship_Count","Months_Inactive_12_mon","Contacts_Count_12_mon","Credit_Limit","Total_Revolving_Bal","Avg_Open_To_Buy","Total_Amt_Chng_Q4_Q1","Total_Trans_Amt","Total_Trans_Ct","Total_Ct_Chng_Q4_Q1","Avg_Utilization_Ratio","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"

768805383,"Existing Customer",45,"M",3,"High School","Married","$60K - $80K","Blue",39,5,1,3,12691,777,11914,1.335,1144,42,1.625,0.061,9.3448e-05,0.99991

818770008,"Existing Customer",49,"F",5,"Graduate","Single","Less than $40K","Blue",44,6,1,2,8256,864,7392,1.541,1291,33,3.714,0.105,5.6861e-05,0.99994

713982108,"Existing Customer",51,"

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("https://gist.githubusercontent.com/sdukshis/c4fa70ed0bd9468f6401ab8dc1e36f8d/raw/e62762bbb28d67b72ad1c4819b65b2fc67ae4b12/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Построение схемы данных

Разделим признаки на следующие категории:
* числовые
* категориальные
* текстовые


In [3]:
num_feat = [
    "Age",
    "SibSp",
    "Parch",
    "Fare",
]

cat_feat = [
    "Pclass",
    "Sex",
    "Embarked",
]

txt_feat = [
    "Name",
    "Ticket",
    "Cabin",
]



Создадим класс для работы со схемой

In [4]:
from typing import List
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


class Schema:

    def __init__(self, num_feat: List[str], cat_feat: List[str], txt_feat: List[str]):
        self._num_feat = num_feat
        self._cat_feat = cat_feat
        self._txt_feat = txt_feat
        self._all_feat = set(self._num_feat + self._cat_feat + self._txt_feat)

    def fit(self, df: pd.DataFrame) -> "Schema":
        self._cat_classes_ = {
            feat: set(df[feat].unique())
            for feat in self._cat_feat
        }

        self._num_minmax = MinMaxScaler().fit(df[self._num_feat])

        return self

    def validate(self, df: pd.DataFrame) -> None:
        self._validate_feat(df)
        self._validate_cat(df)
        self._validate_num(df)

    def _validate_feat(self, df: pd.DataFrame) -> None:
        missing_feat = self._all_feat - set(df.columns)
        if missing_feat:
          raise ValueError(f"Missing {missing_feat} features")

    def _validate_cat(self, df: pd.DataFrame) -> None:
        new_values = dict()
        for feat, classes_ in self._cat_classes_.items():
            new_cat_values = set(df[feat].unique()) - classes_
            if new_cat_values:
              new_values[feat] = new_cat_values

        if new_values:
          raise ValueError(f"Following categorical features includes a new values: {new_values}")


    def _validate_num(self, df:pd.DataFrame) -> None:
        out_of_range = list()
        for i, feat in enumerate(self._num_minmax.feature_names_in_):
            if any((df[feat] < self._num_minmax.data_min_[i]) |
                   (df[feat] > self._num_minmax.data_max_[i])):
                out_of_range.append(feat)

        if out_of_range:
            raise ValueError(f"Following numerical features has out of range values: {out_of_range}")


titanic_schema = Schema(num_feat, cat_feat, txt_feat).fit(train)

In [5]:
titanic_schema

In [8]:
test = pd.read_csv("https://gist.githubusercontent.com/sdukshis/b69647ddf6b694edda41668de2edbe41/raw/69b835e48670f8648925f8d1312b9f278b56deea/test.csv")
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [12]:
test.drop(["Sex", "Name"], axis=1)

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,47.0,1,0,363272,7.0000,NaN,S
2,894,2,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,27.0,0,0,315154,8.6625,NaN,S
4,896,3,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,NaN,0,0,359309,8.0500,NaN,S


In [16]:
titanic_schema.validate(test)

ValueError: Following numerical features has out of range values: ['Age', 'Parch']

In [17]:
example_input = test.iloc[0:1, :].copy()

example_input["Embarked"] = "X"

titanic_schema.validate(example_input)

ValueError: Following categorical features includes a new values: {'Embarked': {'X'}}

In [ ]:
titanic_schema.validate(test)

ValueError: ignored

## Выбросы в числовых признаках

In [ ]:
from typing import Tuple

class NumericFeatureOutlierTransformer:

    def __init__(self, quan_range: Tuple[float] = (0.25, 0.75), strategy: str = "clip"):
        self._fitted = False
        self._quan_range = quan_range
        self.transform = {
            "clip": self._clip,
            "nan": self._nan,
        }[strategy]

    def fit(self, x: np.ndarray) -> "NumericFeatureOutlierTransformer":
        lower_quan = np.nanquantile(x, self._quan_range[0])
        upper_quan = np.nanquantile(x, self._quan_range[1])

        iqr = upper_quan - lower_quan
        self._lower_extreme = lower_quan - 1.5*iqr
        self._upper_extreme = upper_quan + 1.5*iqr

        self._fitted = True
        return self

    def _clip(self, x: np.ndarray) -> np.ndarray:
        return x.clip(self._lower_extreme, self._upper_extreme)

    def _nan(self, x:np.ndarray) -> np.ndarray:
        raise NotImplemented

In [ ]:
age_transformer = NumericFeatureOutlierTransformer().fit(train["Age"])

In [ ]:
test["Age_clipped"] = age_transformer.transform(test["Age"])

In [ ]:
test[test["Age"].notna() & (test["Age_clipped"] != test["Age"])][["Age", "Age_clipped"]]

,Age,Age_clipped
81,67.0,64.8125
96,76.0,64.8125


## Заполнение пропусков

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer

age_simple_imputer = SimpleImputer(strategy="mean").fit(train[["Age"]])


In [ ]:
train["Age_simple_imputed"] = age_simple_imputer.transform(train[["Age"]])

Заполняем пропущенный возраст в зависимости от обращения

In [ ]:
train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=True)
train['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: Title, dtype: int64

Заменим редкие обращения

In [ ]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',
           'Don': 'Mr', 'Mme': 'Mrs', 'Jonkheer': 'Mr', 'Lady': 'Mrs',
           'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
train.replace({'Title': mapping}, inplace=True)
train['Title'].value_counts()

Mr        525
Miss      185
Mrs       128
Master     40
Dr          7
Rev         6
Name: Title, dtype: int64

In [ ]:
title_ages = dict(train.groupby('Title')['Age'].median())
title_ages

{'Dr': 46.5, 'Master': 3.5, 'Miss': 21.0, 'Mr': 30.0, 'Mrs': 35.0, 'Rev': 46.5}

In [ ]:
train['age_med'] = train['Title'].apply(lambda x: title_ages[x])


train["Age_imputed" ] = train['Age'].fillna(train['age_med'])